# Описание "Финальный проект - SQL"

<b> Легенда: </b>

"Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта."


<b> Задание: </b>

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


<b> Описание данных: </b>

Таблица `books` - Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors` - Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers` - Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings` - Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews` - Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

# Подготовка к работе 

In [1]:
#pip install psycopg2

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format\
(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

# Знакомство с таблицами

## Таблица books

In [3]:
query = (
'''
SELECT *
FROM books
LIMIT 5;

''')

pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# проверим книги на уникальность
query = (
'''
SELECT title,
       COUNT(*)
FROM books
GROUP BY title
HAVING COUNT(*) > 1;

''')

pd.io.sql.read_sql(query, con = engine) 

,title,count
0,Memoirs of a Geisha,2


In [5]:
# посмотрим на обнаруженный "дубликат"
query = (
'''
SELECT *
FROM books
WHERE title = 'Memoirs of a Geisha';

''')

pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


Разные издания одной и той же книги. Пока оставим как есть. По мере необходимости - будем иметь это в виду. 

## Таблица authors

In [6]:
query = (
'''
SELECT *
FROM authors
LIMIT 5;

''')

pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


## Таблица publishers

In [7]:
query = (
'''
SELECT *
FROM publishers
LIMIT 5;

''')

pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Таблица ratings

In [8]:
query = (
'''
SELECT *
FROM ratings
LIMIT 5;

''')

pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


## Таблица reviews

In [9]:
query = (
'''
SELECT *
FROM reviews
LIMIT 5;

''')

pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Решение

## Количество книг, вышедших после 1 января 2000 года

In [10]:
query = (
'''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01';

''')

pd.io.sql.read_sql(query, con = engine) 

,count
0,819


<b> Вывод: </b>

После 1 января 2000 года вышло 819 книг.

## Количество обзоров и средняя оценка каждой книги

In [11]:
query = (
'''
SELECT t_review.book_id AS book_id,
       t_review.book_title AS book_title,
       t_review.cnt_review AS cnt_review,
       t_rating.avg_rating AS avg_rating
FROM
  (SELECT b.title AS book_title,
          b.book_id,
          COUNT(rew.review_id) AS cnt_review
   FROM books AS b
   LEFT JOIN reviews AS rew ON b.book_id = rew.book_id
   GROUP BY b.book_id
   ORDER BY cnt_review DESC) AS t_review
JOIN
  (SELECT rat.book_id,
          ROUND(AVG(rat.rating), 2) AS avg_rating
   FROM ratings AS rat
   GROUP BY rat.book_id) AS t_rating ON t_review.book_id = t_rating.book_id
ORDER BY cnt_review DESC;

''')

pd.io.sql.read_sql(query, con = engine)

,book_id,book_title,cnt_review,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,695,The Curious Incident of the Dog in the Night-Time,6,4.08
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,696,The Da Vinci Code (Robert Langdon #2),6,3.83
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
996,808,The Natural Way to Draw,0,3.00
997,221,Essential Tales and Poems,0,4.00
998,387,Leonardo's Notebooks,0,4.00


In [12]:
query = (
'''
SELECT COUNT(book_title)
FROM
  (SELECT b.title AS book_title
   FROM books AS b
   JOIN ratings AS rat ON b.book_id = rat.book_id
   GROUP BY book_title
   HAVING ROUND(AVG(rat.rating), 2) = '5.00') AS t;

''')

pd.io.sql.read_sql(query, con = engine)

,count
0,44


<b> Вывод: </b>

Больше всего обзоров на книгу "Twilight (Twilight #1)" - 7 обзоров.

Самой высокой оценкой "5" обладают 44 книги.

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [13]:
query = (
'''
SELECT publisher,
       COUNT(title) AS cnt_book
FROM
  (SELECT *
   FROM books AS b
   JOIN publishers AS p ON b.publisher_id = p.publisher_id
   WHERE num_pages > 50) AS t
GROUP BY publisher
ORDER BY cnt_book DESC
LIMIT 1;

''')

pd.io.sql.read_sql(query, con = engine) 

,publisher,cnt_book
0,Penguin Books,42


<b> Вывод: </b>

Наибольшее число книг толще 50 страниц выпустило издательство "Penguin Books" - 42 книги.

## Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками)


In [14]:
# id книг с 50 и больше оценками
query = (
'''
SELECT book_id,
       COUNT(username) AS cnt_rating 
FROM ratings
GROUP BY book_id
HAVING COUNT(username) >= 50
ORDER BY cnt_rating DESC;

''')

pd.io.sql.read_sql(query, con = engine)

,book_id,cnt_rating
0,948,160
1,750,88
2,673,86
3,75,84
4,302,82
5,299,80
6,301,75
7,79,74
8,722,74
9,300,73


In [15]:
# средний рейтинг книг
query = (
'''
SELECT b.title AS book_title,
       ROUND(AVG(rat.rating),2) AS avg_rating,
       a.author
FROM books AS b
JOIN ratings AS rat ON b.book_id = rat.book_id
JOIN authors AS a ON b.author_id = a.author_id
GROUP BY book_title,
         a.author
ORDER BY avg_rating DESC;

''')

pd.io.sql.read_sql(query, con = engine)

,book_title,avg_rating,author
0,Pop Goes the Weasel (Alex Cross #5),5.00,James Patterson
1,The Woman in Black,5.00,Susan Hill/John Lawrence
2,Light in August,5.00,William Faulkner
3,The Demon-Haunted World: Science as a Candle i...,5.00,Carl Sagan/Ann Druyan
4,A Dirty Job (Grim Reaper #1),5.00,Christopher Moore
...,...,...,...
994,The World Is Flat: A Brief History of the Twen...,2.25,Thomas L. Friedman
995,Junky,2.00,William S. Burroughs/Oliver Harris/Allen Gins...
996,His Excellency: George Washington,2.00,Joseph J. Ellis
997,Drowning Ruth,2.00,Christina Schwarz


In [16]:
# книга / средний рейтинг книги / автор
query = (
'''
SELECT b.title AS book_title,
       ROUND(AVG(rat.rating), 2) AS avg_rating,
       a.author
FROM
  (SELECT book_id,
          COUNT(username) AS cnt_rating
   FROM ratings
   GROUP BY book_id
   HAVING COUNT(username) >= 50) AS t
JOIN books AS b ON b.book_id = t.book_id
JOIN ratings AS rat ON b.book_id = rat.book_id
JOIN authors AS a ON b.author_id = a.author_id
GROUP BY book_title,
         a.author
ORDER BY avg_rating DESC;

''')

pd.io.sql.read_sql(query, con = engine)

,book_title,avg_rating,author
0,Harry Potter and the Prisoner of Azkaban (Harr...,4.41,J.K. Rowling/Mary GrandPré
1,The Fellowship of the Ring (The Lord of the Ri...,4.39,J.R.R. Tolkien
2,Harry Potter and the Chamber of Secrets (Harry...,4.29,J.K. Rowling/Mary GrandPré
3,The Book Thief,4.26,Markus Zusak/Cao Xuân Việt Khương
4,Harry Potter and the Half-Blood Prince (Harry ...,4.25,J.K. Rowling/Mary GrandPré
5,Harry Potter and the Order of the Phoenix (Har...,4.19,J.K. Rowling/Mary GrandPré
6,Little Women,4.19,Louisa May Alcott
7,The Hobbit or There and Back Again,4.13,J.R.R. Tolkien
8,The Lightning Thief (Percy Jackson and the Oly...,4.08,Rick Riordan
9,Lord of the Flies,3.90,William Golding


In [17]:
# автор / средний рейтинг всех книг
query = (
'''
WITH i AS
  (SELECT b.title AS book_title,
          ROUND(AVG(rat.rating), 2) AS avg_rating,
          a.author
   FROM
     (SELECT book_id,
             COUNT(username) AS cnt_rating
      FROM ratings
      GROUP BY book_id
      HAVING COUNT(username) >= 50) AS t
   JOIN books AS b ON b.book_id = t.book_id
   JOIN ratings AS rat ON b.book_id = rat.book_id
   JOIN authors AS a ON b.author_id = a.author_id
   GROUP BY book_title,
            a.author
   ORDER BY avg_rating DESC)
   
SELECT author,
       AVG(avg_rating) AS avg_rating
FROM i
GROUP BY author
ORDER BY avg_rating DESC
LIMIT 1;

''')

pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.285


<b> Вывод: </b>

J.K. Rowling/Mary GrandPré является автором с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками).

## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [18]:
# пользователи, которые поставили больше 50 оценок
query = (
'''
SELECT rat.username,
       COUNT(rat.book_id) AS cnt_rating
FROM ratings AS rat
GROUP BY rat.username
HAVING COUNT(rat.book_id) > 50
ORDER BY cnt_rating DESC;

''')

pd.io.sql.read_sql(query, con = engine) 

,username,cnt_rating
0,paul88,56
1,martinadam,56
2,sfitzgerald,55
3,richard89,55
4,jennifermiller,53
5,xdavis,51


In [19]:
# количество обзоров каждого пользователя, которые поставили больше 50 оценок
query = (
'''
SELECT COUNT(rew.book_id) AS cnt_review,
       rew.username AS username
FROM
  (SELECT rat.username,
          COUNT(rat.book_id) AS cnt_rating
   FROM ratings AS rat
   GROUP BY rat.username
   HAVING COUNT(rat.book_id) > 50
   ORDER BY cnt_rating DESC) AS rat
JOIN reviews AS rew ON rat.username = rew.username
GROUP BY rew.username;

''')

pd.io.sql.read_sql(query, con = engine) 

,cnt_review,username
0,28,sfitzgerald
1,25,jennifermiller
2,18,xdavis
3,22,paul88
4,27,martinadam
5,26,richard89


In [20]:
# среднее количество обзоров от пользователей, которые поставили больше 50 оценок
query = (
'''
WITH i AS
  (SELECT COUNT(rew.book_id) AS cnt_review,
          rew.username AS username
   FROM
     (SELECT rat.username,
             COUNT(rat.book_id) AS cnt_rating
      FROM ratings AS rat
      GROUP BY rat.username
      HAVING COUNT(rat.book_id) > 50
      ORDER BY cnt_rating DESC) AS rat
   JOIN reviews AS rew ON rat.username = rew.username
   GROUP BY rew.username)

SELECT ROUND(AVG(cnt_review)) AS avg_review
FROM i;

''')

pd.io.sql.read_sql(query, con = engine) 

,avg_review
0,24.0


<b> Вывод: </b>

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24 обзора.

# Сводные результаты исследования

- После 1 января 2000 года вышло 819 книг;
- Самой высокой оценкой "5" обладают 44 книги;
- Больше всего обзоров на книгу "Twilight (Twilight #1)" - 7 обзоров;
- Наибольшее число книг толще 50 страниц выпустило издательство "Penguin Books" - 42 книги;
- J.K. Rowling/Mary GrandPré является автором с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками);
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24 обзора.